In [42]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [44]:
import os
import sys

In [46]:
import pandas as pd
import plotly.graph_objects as go
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

In [48]:
path = os.getcwd()
py_files = os.path.abspath(os.path.join(path, os.pardir))
sys.path.append(py_files)

In [50]:
import BrainMapping as brm
import UniversalTools as uto

In [52]:
path = os.getcwd()
data_folder = path
receptor_density_file = "/ReceptorDensity.csv"

In [54]:
# Function to generate the bar/line chart
def graph_index_columns(df: pd.DataFrame, barmode: str = 'group', chart_type: str = 'bar') -> go.Figure:
    col_names = df.columns.values.tolist()
    index_names = df.index.values.tolist()
    index_names = list(map(str, index_names))

    chart_elements = []

    for i in col_names:
        if chart_type == 'bar':
            chart_element = go.Bar(name=i, x=index_names, y=df[i])
        elif chart_type == 'line':
            chart_element = go.Scatter(name=i, x=index_names, y=df[i])
        else:
            raise ValueError("Invalid chart_type. Expect 'bar' or 'line'.")
        chart_elements.append(chart_element)

    fig = go.Figure(chart_elements)
    fig.update_layout(barmode=barmode)

    return fig

In [56]:
receptor_density = pd.read_csv(data_folder + receptor_density_file)

In [58]:
a = brm.SetReceptorDensity(receptor_density,'max',scaled=False)

In [60]:
b = brm.SetReceptorDensity(receptor_density,'max',scaled=True)

In [62]:
df = a.copy()

In [64]:
#df.columns

In [66]:
receptors = [
    'Acetylcholine',
    'Adrenergic',
    'Cannabinoid',
    'Dopamine',
    'GABA',
    'Glutamate',
    'Histamine',
    'Melatonin',
    'Muscarinic',
    'Neuropeptide',
    'Neuropeptide Y',
    'Noradrenaline',
    'Opioid',
    'Orexin',
    'Prostaglandin',
    'Serotonin',
    'Somatostatin'
]

In [68]:
df_receptors = pd.DataFrame(receptors, columns=['Receptor Name'])
df_receptors

,Receptor Name
0,Acetylcholine
1,Adrenergic
2,Cannabinoid
3,Dopamine
4,GABA
5,Glutamate
6,Histamine
7,Melatonin
8,Muscarinic
9,Neuropeptide


In [70]:
number = 3
receptor = receptors[number]
c = brm.ReceptorFilter(df,receptor)
receptor

'Dopamine'

In [72]:
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Receptor Density Across Brain Structures"),
    dcc.Graph(id='receptor-graph', figure=graph_index_columns(c))  # Generate the figure using your function
])

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

In [17]:
def BrainStructureFilter(df, brain_structure):
    # Filter the dataframe based on the brain structure input
    df_new = df[df['Brain structure'].str.contains(brain_structure, case=False)].copy()
    # Transpose the dataframe and set the brain structure as the column name
    df_new = df_new.drop(columns=['Brain structure']).T
    df_new.columns = [brain_structure]
    return df_new

In [18]:
d = BrainStructureFilter(df,"Nucleus Accumbens")
d

,Nucleus Accumbens
H1 Histamine Receptor Density (fmol/mg),1000.0
H2 Histamine Receptor Density (fmol/mg),2500.0
H3 Histamine Receptor Density (fmol/mg),3000.0
α4β2 Nicotinic Receptor Density (fmol/mg),3000.0
α7 Nicotinic Receptor Density (fmol/mg),5000.0
...,...
OX2 Orexin Receptor Density (fmol/mg),2500.0
CB1 Cannabinoid Receptor Density (fmol/mg),9000.0
CB2 Cannabinoid Receptor Density (fmol/mg),3500.0
GABA-A Receptor Density (fmol/mg),10000.0


In [19]:
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Receptor Density Across Brain Structures"),
    dcc.Graph(id='receptor-graph', figure=graph_index_columns(d))  # Generate the figure using your function
])

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, host='0.0.0.0', port=8051)

In [20]:
app = dash.Dash(__name__)

# Layout of the app
app.layout = html.Div([
    html.H1("Receptor Density Across Brain Structures"),
    
    dcc.Dropdown(
        id='structure-dropdown',
        options=[{'label': structure, 'value': structure} for structure in receptors],
        value=receptors[0],  # Default value
        clearable=False,
    ),
    
    dcc.Graph(id='receptor-graph')  # Graph to display the filtered data
])

# Callback to update the graph based on the dropdown selection
@app.callback(
    Output('receptor-graph', 'figure'),
    [Input('structure-dropdown', 'value')]
)
def update_graph(selected_structure):
    # Filter the DataFrame using ReceptorFilter with the selected structure (receptor)
    filtered_df = ReceptorFilter(c, selected_structure)
    
    # Generate the figure using graph_index_columns with the filtered data
    fig = graph_index_columns(filtered_df, colors='large', barmode='group', chart_type='bar')
    
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, host='0.0.0.0', port=8051)